# Tournament to evaluate ELO

##  Library imports

In [9]:
import numpy as np
import pandas as pd
from glob import glob


import sys
sys.setrecursionlimit(199999)

## Path variables

In [10]:
path = "alquerque/best_models/*/*[!best][!temp].h5"

In [11]:
players = glob(path)

In [12]:
len(players)

75

## DataFrame Creation

In [16]:
games_df = (
    pd.merge(
        pd.Series(players).rename("Player_1"),
        pd.Series(players).rename("Player_2"),
        how = "cross"
    )
    .loc[lambda a: 
        (a.Player_1 != a.Player_2) & # Eliminar partidas contra uno mismo
        ~a.apply(frozenset, axis=1).duplicated() # Eliminar duplicados ab == ba
    ]
    # .groupby('Player_1').apply(lambda x: x.sample(10)).reset_index(drop=True) # Select number of games per player
    .sample(n = 10)
)
games_df

,Player_1,Player_2
1526,alquerque/best_models/100_nonsymetric/checkpoi...,alquerque/best_models/100_nonsymetric/checkpoi...
373,alquerque/best_models/100_nonsymetric/checkpoi...,alquerque/best_models/200_nonsymetric_10max/ch...
3974,alquerque/best_models/200_nonsymetric_30max/ch...,alquerque/best_models/200_nonsymetric_10max/ch...
2906,alquerque/best_models/200_nonsymetric_10max_10...,alquerque/best_models/200_nonsymetric_30max/ch...
3956,alquerque/best_models/200_nonsymetric_30max/ch...,alquerque/best_models/200_nonsymetric_30max/ch...
725,alquerque/best_models/100_nonsymetric/checkpoi...,alquerque/best_models/200_nonsymetric_10max_10...
221,alquerque/best_models/100_nonsymetric/checkpoi...,alquerque/best_models/200_nonsymetric_10max/ch...
650,alquerque/best_models/100_nonsymetric/checkpoi...,alquerque/best_models/200_nonsymetric_10max_10...
1296,alquerque/best_models/100_nonsymetric/checkpoi...,alquerque/best_models/100_nonsymetric/checkpoi...
2755,alquerque/best_models/100_nonsymetric/checkpoi...,alquerque/best_models/200_nonsymetric_30max/ch...


## Player function

### Librerías

In [17]:
import Arena
from MCTS import MCTS
from alquerque.AlquerqueGame import AlquerqueGame
from alquerque.AlquerquePlayers import *
from alquerque.keras.NNet import NNetWrapper as NNet

import tensorflow.compat.v2 as tf
physical_devices = tf.config.list_physical_devices('GPU') 
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)


from utils import *

### Juegos

In [18]:
def pit(players):
    
    # Game

    g = AlquerqueGame()

    # Jugadores

    n1 = NNet(g)
    n1.load_checkpoint("/".join(players[0].split("/")[:-1]), players[0].split("/")[-1])
    args1 = dotdict({'numMCTSSims': 5, 'cpuct':1.0})
    mcts1 = MCTS(g, n1, args1)
    n1p = lambda x: np.argmax(mcts1.getActionProb(x, temp=0))

    n2 = NNet(g)
    n2.load_checkpoint("/".join(players[1].split("/")[:-1]), players[1].split("/")[-1])
    args2 = dotdict({'numMCTSSims': 5, 'cpuct': 1.0})
    mcts2 = MCTS(g, n2, args2)
    n2p = lambda x: np.argmax(mcts2.getActionProb(x, temp=0))

    #Juego

    arena = Arena.Arena(n1p, n2p, g)
    
    player_1, player_2, _ =  arena.playGames(2)

    if player_1 > player_2:
        res = 1
    elif player_1 < player_2:
        res = 2
    else:
        res = 0

    return res

## Cálculo de ELO

### Aplicación de partidas

In [19]:
games_df = (
    games_df
    .assign(
        res = lambda a: a.apply(pit, axis = 1)
    )
)
games_df

Arena.playGames (2): 100%|██████████| 1/1 [00:09<00:00,  9.24s/it]


,Player_1,Player_2,res
1526,alquerque/best_models/100_nonsymetric/checkpoi...,alquerque/best_models/100_nonsymetric/checkpoi...,1
373,alquerque/best_models/100_nonsymetric/checkpoi...,alquerque/best_models/200_nonsymetric_10max/ch...,2
3974,alquerque/best_models/200_nonsymetric_30max/ch...,alquerque/best_models/200_nonsymetric_10max/ch...,2
2906,alquerque/best_models/200_nonsymetric_10max_10...,alquerque/best_models/200_nonsymetric_30max/ch...,1
3956,alquerque/best_models/200_nonsymetric_30max/ch...,alquerque/best_models/200_nonsymetric_30max/ch...,2
725,alquerque/best_models/100_nonsymetric/checkpoi...,alquerque/best_models/200_nonsymetric_10max_10...,2
221,alquerque/best_models/100_nonsymetric/checkpoi...,alquerque/best_models/200_nonsymetric_10max/ch...,0
650,alquerque/best_models/100_nonsymetric/checkpoi...,alquerque/best_models/200_nonsymetric_10max_10...,2
1296,alquerque/best_models/100_nonsymetric/checkpoi...,alquerque/best_models/100_nonsymetric/checkpoi...,0
2755,alquerque/best_models/100_nonsymetric/checkpoi...,alquerque/best_models/200_nonsymetric_30max/ch...,2


### Cálculo de ELO

In [20]:
elo_dict = (
    pd.DataFrame(
        pd.Series(
            np.concatenate(
                [
                    games_df.Player_1, 
                    games_df.Player_2
                ]
            )
        )
        .rename("Player")
        .drop_duplicates()
    )
    .assign(
        ELO = 1000
    )
    .set_index("Player")
    .to_dict('index')
)
elo_dict

{'alquerque/best_models/100_nonsymetric/checkpoint_48.h5': {'ELO': 1000},
 'alquerque/best_models/100_nonsymetric/checkpoint_35.h5': {'ELO': 1000},
 'alquerque/best_models/200_nonsymetric_30max/checkpoint_2.h5': {'ELO': 1000},
 'alquerque/best_models/200_nonsymetric_10max_100MCTS/checkpoint_2.h5': {'ELO': 1000},
 'alquerque/best_models/100_nonsymetric/checkpoint_6.h5': {'ELO': 1000},
 'alquerque/best_models/100_nonsymetric/checkpoint_47.h5': {'ELO': 1000},
 'alquerque/best_models/100_nonsymetric/checkpoint_49.h5': {'ELO': 1000},
 'alquerque/best_models/100_nonsymetric/checkpoint_88.h5': {'ELO': 1000},
 'alquerque/best_models/100_nonsymetric/checkpoint_43.h5': {'ELO': 1000},
 'alquerque/best_models/100_nonsymetric/checkpoint_11.h5': {'ELO': 1000},
 'alquerque/best_models/200_nonsymetric_10max/checkpoint_19.h5': {'ELO': 1000},
 'alquerque/best_models/200_nonsymetric_10max/checkpoint_90.h5': {'ELO': 1000},
 'alquerque/best_models/200_nonsymetric_30max/checkpoint_6.h5': {'ELO': 1000},
 'al

In [21]:
for _, row in games_df.iterrows():

    K = 20 # Development coefficient

    rat_1 = elo_dict[row["Player_1"]]["ELO"]
    rat_2 = elo_dict[row["Player_2"]]["ELO"]

    expected_score_1 = 1 / (1 + 10**((rat_2 - rat_1)/400))
    expected_score_2= 1 / (1 + 10**((rat_1 - rat_2)/400))

    if row["res"] == 1:

        rat_1_new = rat_1 + K * (1 - expected_score_1)
        rat_2_new = rat_2 + K * (0 - expected_score_2)

    elif row["res"] == 2:
        
        rat_2_new = rat_2 + K * (1 - expected_score_2)
        rat_1_new = rat_1 + K * (0 - expected_score_1)
        
    else:
        
        rat_2_new = rat_2 + K * (0.5 - expected_score_2)
        rat_1_new = rat_1 + K * (0.5 - expected_score_1)

    # Actualizar valores ELO

    elo_dict[row["Player_1"]]["ELO"] = rat_1_new
    elo_dict[row["Player_2"]]["ELO"] = rat_2_new

In [31]:
elo_df = pd.DataFrame.from_dict(elo_dict, orient = "index")
elo_df

,ELO
alquerque/best_models/100_nonsymetric/checkpoint_11.h5,990.000000
alquerque/best_models/100_nonsymetric/checkpoint_35.h5,990.000000
alquerque/best_models/100_nonsymetric/checkpoint_36.h5,1000.000000
alquerque/best_models/100_nonsymetric/checkpoint_43.h5,990.000000
alquerque/best_models/100_nonsymetric/checkpoint_47.h5,1000.000000
alquerque/best_models/100_nonsymetric/checkpoint_48.h5,1010.000000
alquerque/best_models/100_nonsymetric/checkpoint_49.h5,990.287744
alquerque/best_models/100_nonsymetric/checkpoint_6.h5,990.000000
alquerque/best_models/100_nonsymetric/checkpoint_88.h5,1000.000000
alquerque/best_models/200_nonsymetric_10max/checkpoint_19.h5,1010.000000


In [32]:
elo_df.to_csv("ELO.csv", sep = ";")